In [1]:
/**
 *This code focuses on calibrating and debugging the existing open-source Kalman Filter from apache.commons-math. The current implementation exhibits two main limitations:

 Lack of native Kotlin support: Although the Java-based library can be used in Kotlin, it does not leverage Kotlin-specific features or idioms.

 Limited flexibility in system modeling: In the update step, the filter always assumes the same measurement covariance. In real-world applications, this covariance can change from one iteration to the next, reducing modeling accuracy when using a fixed value.
 * 
 */

In [2]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [3]:
%use kandy
%use dataframe

In [4]:
import kotlin.random.Random

// Prepare the measurement data 
fun getRandomMeasure(
    trueValue: Double,
    range: Double
): DoubleArray {

    val measureVal = Random.nextDouble(trueValue - range, trueValue + range)
    return  doubleArrayOf(measureVal)
}

In [5]:
import org.apache.commons.math3.filter.KalmanFilter
import org.apache.commons.math3.filter.DefaultProcessModel
import org.apache.commons.math3.filter.ProcessModel
import org.apache.commons.math3.filter.DefaultMeasurementModel
import org.apache.commons.math3.filter.MeasurementModel


val processModel: ProcessModel = DefaultProcessModel(
    arrayOf( //stateTransition: X
        doubleArrayOf(1.0)
    ),
    arrayOf( // control : U
        doubleArrayOf(1.0)
    ),
    arrayOf( // processNoise: Q
        doubleArrayOf(1.0)
    ),
    doubleArrayOf(50.0), // inital state estimate
    arrayOf( // initial covariance
        doubleArrayOf(100.0*100.0)
    ),
)

val measurementModel = DefaultMeasurementModel(
    arrayOf( //measMatrix: H
        doubleArrayOf(1.0)
    ),
    arrayOf( //measNoise : R
        doubleArrayOf(100.0*100)
    ),
)

val kalmanFilter = KalmanFilter(processModel, measurementModel)


In [6]:

val trueValue = 100.0
val range = 100.0


val estimate = mutableListOf<Double>()
val measurement = mutableListOf<Double>()
val covariances = mutableListOf<Double>()


repeat(100){ i ->
    
    kalmanFilter.predict()

    kalmanFilter.stateEstimation
    
    
    kalmanFilter.correct(
            getRandomMeasure(
                trueValue = trueValue ,
                range = range
            ).also {
                measurement.add(it[0])
            }
        )
    
    estimate.add(kalmanFilter.stateEstimation[0])
    covariances.add(kalmanFilter.errorCovariance[0][0])

}


val plotData = dataFrameOf(
    "data" to estimate +  measurement,
    "type" to  List(estimate.size) { "estimate" } + List(measurement.size) { "measurement" },
    "index" to List(estimate.size) { it } +  List(measurement.size) { it }
)


plotData.plot {
    line {
        x("index")
        y("data")
        color("type"){
            scale = categorical("measurement" to Color.BLUE, "estimate" to Color.RED )
        }
    }
    layout.size = 1000 to 450

}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7dfnqh"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"data":[53.893550930610644,82.1518181286637,90.6297840348512,108.07436129455861,112.26404571945405,110.58645328168173,108.22343591979391,106.49732997615332,106.94785204577775,97.35193210228559,89.47963479015952,89.61867894426676,93.04689183118934,94.2077635492227,90.75740138743211,89.13603015670145,89.55432722374768,92.96718252461575,92.6500500305655,94.43909700705088,95.88161220970643,93.97546580859995,90.83862829755887,93.82970655904795,97.9919596548478,100.24893512220015,97.10157972191843,100.05320190074471,100.65425383793833,99.32264249749315,98.68655651540884,101.66978081782085,104.51995953357236,104.11189346212218,105.89265080286938,107.13003467308958,105.86377856284368,105.98041856337706,104.82371080994717,107.25004589823695,106.86441985004473,107.85519995078371,106.92634673307376,106.84694207794053,106.56331268828508,107.12637814144176,105.48301124533373,107.09154917198829,108.13630629496801,106.37308036142717,107.02473601947678,104.83642352668645,106.61790858494066,107.87359436589254,107.39232748278997,108.67110784256073,109.06239158714894,107.61356138428164,106.71399690689505,108.39711108221063,109.15881748351782,109.80330351031037,108.1648972952176,107.6055686834035,106.69036589279546,108.15439158577995,108.81037212111947,108.7492933750205,108.0541000790829,109.45133695462226,110.5694410721618,111.05137586333835,112.02132533617906,111.32336432209615,111.37282583848254,111.15819932999874,110.83178490272111,111.82056520400188,111.97248794333039,113.08500829692686,113.95895316237328,113.26060123974841,112.58843900126678,112.33775144845343,112.5717428330839,112.70794930185986,113.67262232245126,112.92808204275269,111.6324454017093,110.46741492053862,110.56671275653689,111.38436358780523,111.2900869468366,111.07186517151257,111.6871033720269,112.24385290540619,111.35885161961909,111.82533694997962,111.9527428841686,111.18589143859282,57.78671254505985,138.65422706378502,116.05181853272992,177.80037947976405,133.18945265776242,100.53613883839871,91.71532203423726,92.72359317045733,110.98975710421695,1.7605499155740167,3.2806026664253807,91.27822714457186,137.33492332573047,110.34312285827286,39.42573767274779,63.43403256653237,96.59161552698343,153.68853280879117,86.70169024133234,129.71507187764948,125.70548824024816,52.74926620482419,20.018413958275083,164.18432559040502,199.80694405000887,157.56898691852246,14.240061330486764,180.49338431139833,117.58785519639731,60.58818012948643,79.60515087483881,193.85846110602492,195.15823799760304,90.77071502365044,165.69142116508607,149.7719632479085,61.11928731945846,110.2033113158369,61.94849374968634,199.26210086732397,91.9131848478202,147.10374619780814,69.35469794326798,103.56924466919138,94.6223991329648,131.2905812496223,33.62937514961719,178.71057636605963,155.48260356882182,25.080522562917285,137.57670251780783,0.5514862782885821,192.87710349398296,169.6241956287038,83.36446224636637,173.46466521893026,129.17541574650457,32.08678939318887,59.17318161039582,198.5451895998985,150.4922212168692,145.22456822314328,16.989114708581287,76.09860686082382,54.52090812643671,192.5828072038949,147.07140575101977,105.14707722661223,66.60736041255153,193.6401162400307,178.64056423937544,140.69049844984184,172.2668984304375,67.54986105240974,114.50437553146587,97.44316165992295,89.7834819333514,176.14799587922582,121.94224463065831,186.7148589023695,172.2812985324189,66.2762719958241,67.00519714340936,95.20438347260153,128.6862472603344,122.15838631191953,181.09483168021754,60.518226277807855,19.789733030344237,27.315588421665527,117.70156692796276,170.52190017613583,104.42746480507175,9

In [7]:
val convarianceDf = dataFrameOf(
    "covariances" to covariances,
    "index" to List(covariances.size) { it } 
)


convarianceDf.plot {
    line {
        x("index")
        y("covariances")
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UHeTYc"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"covariances":[5000.249987500624,3333.8888370420977,2500.8748687715583,2001.1997360659825,1668.1939811828634,1430.4278289267613,1252.1863850678317,1113.6280358163442,1002.8478076139929,912.2698034040341,836.843420767596,773.0720847674465,718.4582381454844,671.1703354422079,629.8347143222918,593.4009240802189,561.0519446448299,532.1427574779174,506.1573452039441,482.6779770605026,461.36287104472865,441.9296833264578,424.14312530865226,407.8055525938549,392.74972572811805,378.83317966898085,365.9337997964761,353.9463132326356,342.7794818975044,332.35383884502545,322.59984903631727,313.4564045181628,304.86958515857197,296.7916318279955,289.180090717308,281.9970964153263,275.20876818489006,268.7846991198072,262.6975219285071,256.92253826178563,251.43740099395492,246.22184083706924,241.25743023523174,236.52737873985254,232.01635507527212,227.71033191958392,223.5964500880168,219.6628993470773,215.89881353113978,212.29417799839769,208.83974776510266,205.52697490774605,202.34794403177756,199.29531478020547,196.3622705020883,193.5424723244345,190.83001797537003,188.2194047948589,185.7054964444241,183.28349289138865,180.9489032979278,178.69752149217197,176.52540373894266,174.4288485624656,172.40437840342162,170.44872291868447,168.55880375464102,166.73172064459595,164.96473869784833,163.25527676294757,161.6008967606908,159.99929389386875,158.44828765082127,156.9458135287095,155.4899154102119,154.0787385342384,152.71052300735005,151.3835978079703,150.09637524026562,148.84734579883184,147.63507340911468,146.45819101187308,145.31539646301218,144.2054487228117,143.12716431099156,142.07941400622312,141.06111977063864,140.07125188164053,139.10882625488432,138.17290194372762,137.2625788017162,136.37699529583304,135.51532645928202,134.67678197352194,133.860604370125,133.0660673438108,132.2924741687126,131.53915621057536,130.80547152816837,130.0908035577279],
"index":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"index",
"y":"covariances"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("UHeTYc");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.682003994035554 15.500000000000057 L23.682003994035554 15.500000000000057 L28.466247225153847 121.56880332480736 L33.25049045627214 174.59259996276447 L38.034733687390435 206.39839630088775 L42.818976918508724 227.59519503266762 L47.60322014962702 242.7297120813933 L52.38746338074532 254.07530583086367 L57.171706611863605 262.8949535355176 L61.9559498429819 269.9464418843903 L66.7401930741002 275.7119990755526 L71.52443630521849 280.51311085325136 L76.30867953633678 284.5723445278355 L81.09292276745508 288.04867698707585 L85.87716599857336 291.05869150289453 L90.66140922969166 293.6898257355131 L95.44565246080995 296.00894397137716 L100.22989569192825 298.06805194838824 L105.014138923046